In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f, Cylinder
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../functions.jl")

circle_center_tangent_to_lines (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [12]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

In [13]:
add_coordinate!(robot, FrameOrigin("rh_fftip"); id = "rh_fftip")
add_coordinate!(robot, FrameOrigin("rh_ffdistal"); id="rh_ffdistal")
add_coordinate!(robot, FrameOrigin("rh_ffmiddle"); id="rh_ffmiddle")
add_coordinate!(robot, FrameOrigin("rh_ffproximal"); id="rh_ffproximal")
add_coordinate!(robot, FrameOrigin("rh_thtip"); id = "rh_thtip")
add_coordinate!(robot, FrameOrigin("rh_thdistal"); id="rh_thdistal")
add_coordinate!(robot, FrameOrigin("rh_thmiddle"); id="rh_thmiddle")

"rh_thmiddle"

### Hand Visualization

In [16]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.033, -0.01, 0.38]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.033
 0.0  1.0  0.0  -0.01
 0.0  0.0  1.0   0.38
 0.0  0.0  0.0   1.0

In [128]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

box_dimensions = [0.02, 0.05, 0.05] 
# box_position = SVector(0.053, -0.03, 0.37)
box_position = SVector(0.042 + box_dimensions[1], -0.03, 0.32+box_dimensions[3])

box_visual_dimensions = box_dimensions - 0.004*SVector(1.,1.,1.)
box = Rect3f(Point3f(box_position) - Vec3f(box_visual_dimensions), 2*Vec3f(box_visual_dimensions))
mesh!(ls, box; color=:magenta, transparency=true)


# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_thtip")
display_frame(m, ls, "world")
#display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [ ]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), coordinates(robot))

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [14]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 0.01, (limits.lower+0.0, limits.upper-0.0), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.0001, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Creation of the box prismatic joints

In [15]:
vm = Mechanism{Float64}("VirtualBox")

# box_position = SVector(0.055, -0.01, 0.37)
# box_dimensions = [0.01, 0.05, 0.05] 

box_dimensions = [0.019, 0.065, 0.07] 
# box_position = SVector(0.053, -0.03, 0.37)
box_position = SVector(0.042 + box_dimensions[1], -0.03, 0.32+box_dimensions[3])


# box_position = SVector(0.0, -0.065, 0.32)
# box_dimensions = [0.03, 0.02, 0.03]


attracted_frames = ("rh_fftip", "rh_ffdistal", "rh_ffmiddle","rh_ffproximal", "rh_thtip", "rh_thdistal")
attracted_frames_names = ("fftip", "ffdistal", "ffmiddle", "ffprox", "thtip", "thdistal")
orientation = (-1.0, -1.0, -1.0, -1.0, 1.0, 1.0)


for i in 1:length(attracted_frames)
    add_frame!(vm; id = "base_frame_$(attracted_frames_names[i])")
    add_joint!(vm, Rigid(Transform(box_position)); parent=root_frame(vm), child="base_frame_$(attracted_frames_names[i])", id="base_joint_$(attracted_frames_names[i])")
    add_frame!(vm; id = "prism_frame_1_$(attracted_frames_names[i])")
    add_joint!(vm, Prismatic(SVector(0.0,1.0,0.0)); parent="base_frame_$(attracted_frames_names[i])", child="prism_frame_1_$(attracted_frames_names[i])", id="prism_joint_1_$(attracted_frames_names[i])")
    add_frame!(vm; id="prism_frame_2_$(attracted_frames_names[i])")
    add_joint!(vm, Prismatic(SVector(0.0,0.0,1.0)); parent="prism_frame_1_$(attracted_frames_names[i])", child="prism_frame_2_$(attracted_frames_names[i])", id="prism_joint_2_$(attracted_frames_names[i])")
    add_frame!(vm; id="ee_frame_$(attracted_frames_names[i])")
    add_joint!(vm, Rigid(Transform(SVector(orientation[i]*box_dimensions[1],0.0,0.0))); parent ="prism_frame_2_$(attracted_frames_names[i])", child ="ee_frame_$(attracted_frames_names[i])", id = "rigid_joint_$(attracted_frames_names[i])")

    add_coordinate!(vm, FrameOrigin("ee_frame_$(attracted_frames_names[i])"); id="$(attracted_frames_names[i]) ee position")
    add_component!(vm, PointMass(0.01, "$(attracted_frames_names[i]) ee position"); id="$(attracted_frames_names[i]) ee mass")


    joint_damping = 0.05
    add_coordinate!(vm, JointSubspace("prism_joint_1_$(attracted_frames_names[i])"); id="prism_joint_1_$(attracted_frames_names[i])")
    add_component!(vm, LinearDamper(joint_damping, "prism_joint_1_$(attracted_frames_names[i])"); id="prism_joint_1_$(attracted_frames_names[i])_damper")
    add_coordinate!(vm, JointSubspace("prism_joint_2_$(attracted_frames_names[i])"); id="prism_joint_2_$(attracted_frames_names[i])")
    add_component!(vm, LinearDamper(joint_damping, "prism_joint_2_$(attracted_frames_names[i])"); id="prism_joint_2_$(attracted_frames_names[i])_damper")   

    # DEADZONE SPRINGS : CONSTRAINT THE MOTION INSIDE THE BOX
    
    # margin_factor = 0.8
    # deadzone_stiffness = 5.0
    # add_deadzone_springs!(vm, deadzone_stiffness, (-box_dimensions[2]*margin_factor, box_dimensions[2]*margin_factor), "prism_joint_1_$(attracted_frames_names[i])")
    # add_deadzone_springs!(vm, deadzone_stiffness, (-box_dimensions[3]*margin_factor, box_dimensions[3]*margin_factor), "prism_joint_2_$(attracted_frames_names[i])")

    margin = 0.015
    deadzone_stiffness = 5.0
    add_deadzone_springs!(vm, deadzone_stiffness, (-box_dimensions[2] + margin, box_dimensions[2] - margin), "prism_joint_1_$(attracted_frames_names[i])")
    add_deadzone_springs!(vm, deadzone_stiffness, (-box_dimensions[3] + margin, box_dimensions[3] - margin), "prism_joint_2_$(attracted_frames_names[i])")
end


add_gravity_compensation!(vm, VMRobotControl.DEFAULT_GRAVITY)

### Addition of the multiples springs/dampers

Hand motion

In [55]:
vms = VirtualMechanismSystem("myShadowVMS", robot, vm)

D = SMatrix{3, 3}(0.05, 0., 0., 0., 0.05, 0., 0., 0., 0.05)

x_stiffnesses = [0.01, 0.01, 0.01, 0.01, 0.1, 0.1]
yz_stiffness = 0.1

#damping_decay_rate = 161 # 20% of damping at |z| = 0.01
damping_decay_rate = 460 # 20% of damping at |z| = 0.005
exponential_damping_coeff = 0.1
exponential_damping_matrix = SMatrix{3, 3}(exponential_damping_coeff, 0., 0., 0., exponential_damping_coeff, 0., 0., 0., exponential_damping_coeff)

# Establishing contact with the box 

for i in 1:length(attracted_frames)
    K = SMatrix{3, 3}(x_stiffnesses[i], 0., 0., 0., yz_stiffness, 0., 0., 0., yz_stiffness)
    add_coordinate!(vms, CoordDifference(".virtual_mechanism.$(attracted_frames_names[i]) ee position", ".robot.$(attracted_frames[i])"); id = "ee $(attracted_frames_names[i]) diff")
    add_component!(vms, LinearSpring(K, "ee $(attracted_frames_names[i]) diff"); id = "ee $(attracted_frames_names[i]) spring")
    add_component!(vms, LinearDamper(D, "ee $(attracted_frames_names[i]) diff"); id = "ee $(attracted_frames_names[i]) damper")
    add_component!(vms, ExponentialDamper(exponential_damping_matrix, "ee $(attracted_frames_names[i]) diff", damping_decay_rate); id = "ee $(attracted_frames_names[i]) exp damper")
end

add_component!(vms, LinearDamper(SMatrix{3, 3}(1.0, 0., 0., 0., 1.0, 0., 0., 0., 1.0),"ee thdistal diff"); id = "ee thdistal mass damper")

# "Closing" the finger ---> connecting the two extremes to the corners of the box
K = SMatrix{3, 3}(yz_stiffness, 0., 0., 0., yz_stiffness, 0., 0., 0., yz_stiffness)

ext_corner = SVector(box_position[1] - box_dimensions[1], box_position[2] + box_dimensions[2], box_position[3] - box_dimensions[3])
add_coordinate!(vms, ConstCoord(ext_corner); id = "ext corner")
add_coordinate!(vms, CoordDifference(".virtual_mechanism.ffprox ee position", "ext corner"); id = "ext corner diff")
add_component!(vms, LinearSpring(K, "ext corner diff"); id = "ext corner spring")
add_component!(vms, LinearDamper(D, "ext corner diff"); id = "ext corner damper")

int_corner = SVector(box_position[1] - box_dimensions[1], box_position[2] - box_dimensions[2], box_position[3] - box_dimensions[3])
add_coordinate!(vms, ConstCoord(int_corner); id = "int corner")
add_coordinate!(vms, CoordDifference(".virtual_mechanism.fftip ee position", "int corner"); id = "int corner diff")
add_component!(vms, LinearSpring(K, "int corner diff"); id = "int corner spring")
add_component!(vms, LinearDamper(D, "int corner diff"); id = "int corner damper")

# Thumb push 
add_coordinate!(vms,  ConstCoord(box_position);  id="box position")
add_coordinate!(vms, CoordDifference("box position", ".robot.rh_thdistal"); id = "th distal box diff")
add_coordinate!(vms, CoordSlice("th distal box diff", SVector(2)); id = "th distal y diff")
add_component!(vms, LinearSpring(yz_stiffness, "th distal y diff"); id = "th distal y spring")
add_component!(vms, LinearDamper(0.05, "th distal y diff"); id = "th distal y damper")

"th distal y damper"

Box collision model

In [56]:
add_coordinate!(vms, ConstCoord(box_dimensions[1]); id="box dimension 1")
add_coordinate!(vms, ConstCoord(box_dimensions[2]); id="box dimension 2")
add_coordinate!(vms, ConstCoord(box_dimensions[3]); id="box dimension 3")

repulsed_frames = (".robot.rh_fftip_mass_coord", ".robot.rh_ffmiddle_mass_coord", ".robot.rh_ffproximal_mass_coord", ".robot.rh_fftip", 
                    ".robot.rh_ffmiddle", ".robot.rh_ffproximal", ".robot.rh_thtip", ".robot.rh_thdistal", ".robot.rh_thdistal_mass_coord", 
                    ".robot.rh_thproximal_mass_coord", ".robot.rh_thmiddle")
frames_names = ("fftip_mass", "ffmiddle_mass", "ffprox_mass", "fftip", "ffmiddle", "ffprox", "thtip", "thdistal", "thdistal_mass", "thproximal_mass", "thmiddle")

for i in 1:length(repulsed_frames)
    frame = repulsed_frames[i]
    add_coordinate!(vms, CoordDifference(frame, "box position") ; id = "$(frames_names[i]) box diff" )
    for j in 1:3
        add_coordinate!(vms, CoordSlice("$(frames_names[i]) box diff", SVector(j)); id = "$(frames_names[i]) box diff dimension $(j)")
        add_coordinate!(vms, CoordNorm("$(frames_names[i]) box diff dimension $(j)"); id = "$(frames_names[i]) box norm dimension $(j)")
        add_coordinate!(vms, CoordDifference("$(frames_names[i]) box norm dimension $(j)","box dimension $(j)"); id = "shifted $(frames_names[i]) box norm dimension $(j)")

        add_component!(vms, ReLUSpring(0.0, "shifted $(frames_names[i]) box norm dimension $(j)", true); id="$(frames_names[i]) dimension $(j) repulsive spring")
        add_component!(vms, RectifiedDamper(0.0, "$(frames_names[i]) box norm dimension $(j)", (0.0, 1.1*box_dimensions[j]), true, false); id="$(frames_names[i]) dimension $(j) damper")
    end
end

## Simulating the Robot

### Setting Up the Simulation

In [57]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

function setup_box_collision_model(cache, repulsed_frames, frames_names)
    repulsed_frames_coord_ID = []
    repulsive_springs_damper_ID = []
    for i in 1:length(repulsed_frames)
        frame = repulsed_frames[i]
        push!(repulsed_frames_coord_ID, get_compiled_coordID(cache, frame))
        frame_springs_dampers_vec = []
        for j in 1:3
            push!(frame_springs_dampers_vec, get_compiled_componentID(cache, "$(frames_names[i]) dimension $(j) repulsive spring"))
            push!(frame_springs_dampers_vec, get_compiled_componentID(cache, "$(frames_names[i]) dimension $(j) damper"))
        end
        push!(repulsive_springs_damper_ID, frame_springs_dampers_vec)
    end

    return box_position, box_dimensions, repulsed_frames_coord_ID, repulsive_springs_damper_ID
end

function update_box_collision_model(cache, collision_args)
    box_position, box_dimensions, repulsed_frames_coord_ID, repulsive_springs_damper_ID = collision_args
    margin = 0.001
    for i in 1:length(repulsed_frames_coord_ID)
        frame_pos = configuration(cache, repulsed_frames_coord_ID[i])
        for j in 1:3
            # get the indices different from j
            others = filter(x -> x ≠ j, 1:3) 
            #Check if the position of the frame is inside "the field of action" of the spring
            if abs(frame_pos[others[1]] - box_position[others[1]]) < (box_dimensions[others[1]]-margin) && abs(frame_pos[others[2]] - box_position[others[2]]) < (box_dimensions[others[2]]-margin)
                cache[repulsive_springs_damper_ID[i][2*j-1]] = remake(cache[repulsive_springs_damper_ID[i][2*j-1]] ; stiffness = 5.0)
                cache[repulsive_springs_damper_ID[i][2*j]] = remake(cache[repulsive_springs_damper_ID[i][2*j]] ; damping = 5.0)          
            else
                cache[repulsive_springs_damper_ID[i][2*j-1]] = remake(cache[repulsive_springs_damper_ID[i][2*j-1]] ; stiffness = 0.0)
                cache[repulsive_springs_damper_ID[i][2*j]] = remake(cache[repulsive_springs_damper_ID[i][2*j]] ; damping = 0.0)
            end
        end
    end
end

function f_setup(cache)

    box_collision_args = setup_box_collision_model(cache, repulsed_frames, frames_names)
    return box_collision_args
    
end

function f_control(cache, t, args, extra)
    
    collision_args = args 
    update_box_collision_model(cache, collision_args)

end

tspan = (0., 15.)
vms_compiled = compile(vms)

q_init = generate_q_init(vms_compiled; mf=true, rf=true, lf=true)
q_init[21] = 0.2
q_init[23] = -0.7
q_init[24] = -0.26
q_init[5] = 1.57
q_init[4] = 1.0

q = (q_init, zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with box object centric motion"

sol = solve(prob, Rosenbrock23(autodiff=false), progress=true; maxiters=1e6, abstol=1e-3, reltol=1e-3);

[ Info: Simulating shadow robot with box object centric motion
ODE   0%|█                                              |  ETA: N/A
ODE 100%|███████████████████████████████████████████████| Time: 0:00:02


In [58]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.0, -0.2, 0.37]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))

robotvisualize!(ls, plotting_kcache)

box_visual_dimensions = box_dimensions - 0.004*SVector(1.,1.,1.)
box = Rect3f(Point3f(box_position) - Vec3f(box_visual_dimensions), 2*Vec3f(box_visual_dimensions))
mesh!(ls, box; color=:magenta, transparency=true)

plotting_vm_kcache = map(plotting_kcache) do k
    VMRobotControl.virtual_mechanism_cache(k)
end
robotsketch!(ls, plotting_vm_kcache; scale = 0.01)


savepath = joinpath(module_path, "docs/src/assets/shadowhand_object_centric_box_.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);

LoadError: Screen not open!